In [1]:
import os

from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from random import randint
# import librosa

# import tensorflow as tf

In [26]:
file_array = os.listdir("/home/rohan/BE_project/raw/enroll");
for file_name in file_array :
#/home/punit/Downloads/test/{}  directory which contains raw files 
#/home/punit/Downloads/processed/{} directory which contains processed files
#install sox before the script is run. 
#Meaning of parameters refer this article https://digitalcardboard.com/blog/2009/08/25/the-sox-of-silence/
#     os.system("rm -f /home/rohan/BE_project/raw/enroll_processed/*")
# 0.3 -70d -1 0.1 1% - 57
    cmd = "sox /home/rohan/BE_project/raw/enroll/{} /home/rohan/BE_project/raw/enroll_processed/{} silence 1 0.3 -70d -1 0.1 1%".format(file_name, file_name)
#     print(cmd);
    os.system(cmd);

In [27]:
file_array = os.listdir("/home/rohan/BE_project/raw/test");
for file_name in file_array :
#/home/punit/Downloads/test/{}  directory which contains raw files 
#/home/punit/Downloads/processed/{} directory which contains processed files
#install sox before the script is run. 
#Meaning of parameters refer this article https://digitalcardboard.com/blog/2009/08/25/the-sox-of-silence/
#     os.system("rm -f /home/rohan/BE_project/raw/test_processed/*")
    cmd = "sox /home/rohan/BE_project/raw/test/{} /home/rohan/BE_project/raw/test_processed/{} silence 1 0.2 -70d -1 0.1 1%".format(file_name, file_name)
#     print(cmd);
    os.system(cmd);

In [98]:
# every frame is considered independent

no_of_features = 13
no_of_fbank_features = 13
no_of_columns = (3 * no_of_features) + no_of_fbank_features
test_frames = 40
classes = 10

def get_feature_vectors(dataset_type):
    
    #set parameters for training and testing
    if (dataset_type == "train"):
#         directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/enroll')
        directory = os.path.join(os.getcwd(), 'raw/enroll_processed')
        no_of_frames = 800
        start_frame = 10
    elif (dataset_type == "test"):    
#         directory = os.path.join(os.getcwd(), 'data_thuyg20_sre/test')
        directory = os.path.join(os.getcwd(), 'raw/test_processed')
        no_of_frames = test_frames
        start_frame = 1
        
    dataset = numpy.empty([0, no_of_columns + 1])
    
    for file in os.listdir(directory):
        
        # filter speakers
        n = int(classes/2)
#         print(n)
        names = []
        
        for i in range (1, n+1):
            names.append('F0' + '%02d'%(i))
            names.append('M0' + '%02d'%(i))

        if any(name in file for name in names):
            
            # extract mfcc vectors
#             print(file)
            (rate,sig) = wav.read(os.path.join(directory, file))
            fbank_feat = logfbank(sig,rate)
            mfcc_feat = mfcc(sig,rate,winlen=0.032,winstep=0.016,numcep=13,nfft=2048)
            
#             print("Signal :" + str(sig) + " " + file)
#             y, sr = librosa.load(os.path.join(directory, file))
#             mfcc_feat = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=2048).T
            d_mfcc_feat = delta(mfcc_feat, 2)
            dd_mfcc_feat = delta(d_mfcc_feat, 2)
            
#             fbank_feat = logfbank(sig,rate)
            mfcc_vectors = mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            dmfcc_vectors = d_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            ddmfcc_vectors = dd_mfcc_feat[start_frame:start_frame+no_of_frames,:no_of_features]
            fbank_vectors = fbank_feat[start_frame:start_frame+no_of_frames,:no_of_fbank_features]
            
            feature_vectors = numpy.hstack((mfcc_vectors, dmfcc_vectors, ddmfcc_vectors, fbank_vectors))
#             print(feature_vectors.shape)
            
            # get speaker index from filename
            speaker_index = file.split("_")[0]
            if speaker_index[0] == 'M':
                speaker_index = n + int(speaker_index[3:])
            else:
                speaker_index = int(speaker_index[3:])

            #append speaker index to feature vectors
            np_speaker_index = numpy.array([speaker_index])
            temp = numpy.tile(np_speaker_index[numpy.newaxis,:], (feature_vectors.shape[0],1))
            concatenated_feature_vector = numpy.concatenate((feature_vectors,temp), axis=1)
            
#             print(concatenated_feature_vector.shape)
#             print(fbank_vectors.shape)
            
            # append file's data to dataset
            dataset = numpy.concatenate((dataset, concatenated_feature_vector), axis=0)
            

    return dataset


In [99]:
from keras.models import Sequential
import numpy as numpy 

In [100]:
# from numpy import genfromtxt
my_data = get_feature_vectors("train")
# get_feature_vectors("test")

In [101]:
# numpy.set_printoptions(threshold=numpy.nan)
# print(my_data)
print(my_data.shape)

(6400, 53)


In [102]:
Y = numpy.copy(my_data[:, no_of_columns:])
print(Y.shape)

(6400, 1)


In [103]:
X = numpy.copy(my_data[:, :no_of_columns])
print(X.shape)
mean = X.mean(0, keepdims=True)

print(mean.shape)
std_deviation = numpy.std(X, axis=0, keepdims=True)
print(std_deviation.shape)

normalized_X = (X - mean) / std_deviation
print(normalized_X.shape)


(6400, 52)
(1, 52)
(1, 52)
(6400, 52)


In [104]:
from keras import utils as np_utils
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

one_hot_labels = np_utils.to_categorical(Y, num_classes=classes+1)
print(one_hot_labels)

[[ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [105]:
cnn_model = cnn_train()
test_cnn(cnn_model)

Epoch 1/10
6400/6400 [==============================] - 12s 2ms/step - loss: 1.1709 - acc: 0.5605
Epoch 2/10
6400/6400 [==============================] - 15s 2ms/step - loss: 0.7310 - acc: 0.7128
Epoch 3/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.5370 - acc: 0.7980
Epoch 4/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.4226 - acc: 0.8436
Epoch 5/10
6400/6400 [==============================] - 16s 2ms/step - loss: 0.3160 - acc: 0.8897
Epoch 6/10
6400/6400 [==============================] - 14s 2ms/step - loss: 0.2502 - acc: 0.9113
Epoch 7/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.1868 - acc: 0.9339
Epoch 8/10
6400/6400 [==============================] - 12s 2ms/step - loss: 0.1434 - acc: 0.9563
Epoch 9/10
6400/6400 [==============================] - 19s 3ms/step - loss: 0.1205 - acc: 0.9658
Epoch 10/10
6400/6400 [==============================] - 15s 2ms/step - loss: 0.0906 - acc: 0.9770


[array(3.5986104011535645, dtype=float32), array(0.35080644488334656, dtype=float32)]
['loss', 'acc']
6 6 6
2 2 2
4 7 7
9 7 7
2 1 1
3 7 7
7 7 7
3 7 7
1 1 1
1 1 1
7 7 7
10 7 7
6 6 6
10 7 7
9 7 7
1 1 1
1 1 9
3 7 7
4 4 4
1 1 1
9 7 7
4 7 7
4 7 7
2 2 2
6 1 1
2 2 2
9 7 7
10 7 7
7 7 7
3 7 7
7 7 7
2 2 2
10 7 7
6 10 10
3 7 7
6 1 1
4 7 7
7 7 7
7 7 7
3 7 7
7 7 7
4 10 10
1 1 1
4 7 7
9 7 7
3 7 7
6 6 1
1 1 1
10 7 7
4 7 7
9 7 7
10 7 7
10 7 7
7 7 7
9 7 7
1 1 1
2 1 1
3 7 7
6 1 1
10 7 7
6 6 6
9 7 7
Accuracy prob_diff: 25 of 62 - 0.4032258064516129
Accuracy maj_diff: 23 of 62 - 0.3709677419354839


In [106]:
def train():

    # MultiLayer Perceptron
    model = Sequential()

    # 3000 0.6 1500 0.6 -> 57% acc
    # 2560 0.7 1280 0.7 -> cnn_model = cnn_train()
# test_cnn(cnn_model)60% acc
    # 2560 0.7 1600 0.7 -> 51% acc
    # 2560 0.6 1600 0.7 -> 49%
    # 2560 0.7 1280 -   -> 50
    # 3000 0.7 1280 0.7 -> 54% acc
    # 2560 0.7 1280 0.8 -> 45% acc

    model.add(Dense(3000, activation='tanh', input_dim=no_of_columns))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.3))
    
    model.add(Dense(classes+1, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    model.fit(normalized_X, one_hot_labels, epochs=20, batch_size=32)
    
    return model
# score = model.evaluate(x_test, y_test, batch_size=128)

In [107]:
def test(model):
    
    test_model = get_feature_vectors("test")
    print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=classes+1)

#     print(model.test_on_batch(test_X, test_labels, sample_weight=None))
#     print(model.metrics_names)
    predictions = model.predict(test_X)
    
    b = [sum(predictions[current: current+test_frames]) for current in range(0, len(predictions), test_frames)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))
    
    for t, p in zip(test_Y[::test_frames].T[0], predicted_Y):
        print (int(t), p)
    
    diff = predicted_Y - test_Y[::test_frames].T[0]
    correct = sum(x == 0 for x in diff)
    total = len(predicted_Y)
    percent = correct/total * 100
    print(str(correct) + " / " + str(total) + " = " + str(percent))
    
    loss = str(numpy.sum(numpy.abs(diff)))
    print("Loss = " + loss)
    
    return correct, percent, loss

In [108]:
model = train()
test(model)

Epoch 1/20
6400/6400 [==============================] - 4s 621us/step - loss: 1.2456 - acc: 0.5211
Epoch 2/20
6400/6400 [==============================] - 5s 705us/step - loss: 0.8899 - acc: 0.6548
Epoch 3/20
6400/6400 [==============================] - 4s 658us/step - loss: 0.8141 - acc: 0.6975
Epoch 4/20
6400/6400 [==============================] - 4s 637us/step - loss: 0.7322 - acc: 0.7216
Epoch 5/20
6400/6400 [==============================] - 4s 654us/step - loss: 0.6895 - acc: 0.7466
Epoch 6/20
6400/6400 [==============================] - 6s 907us/step - loss: 0.6371 - acc: 0.7641
Epoch 7/20
6400/6400 [==============================] - 7s 1ms/step - loss: 0.6065 - acc: 0.7775
Epoch 8/20
6400/6400 [==============================] - 10s 2ms/step - loss: 0.5722 - acc: 0.7903
Epoch 9/20
6400/6400 [==============================] - 8s 1ms/step - loss: 0.5488 - acc: 0.8017
Epoch 10/20
6400/6400 [==============================] - 7s 1ms/step - loss: 0.5231 - acc: 0.8091
Epoch 11/20
6400

(2480, 53)
6 2
2 2
4 4
9 9
2 2
3 3
7 10
3 3
1 1
1 1
7 10
10 10
6 6
10 10
9 10
1 1
1 1
3 3
4 4
1 1
9 9
4 10
4 4
2 2
6 1
2 1
9 9
10 10
7 10
3 3
7 10
2 6
10 10
6 1
3 3
6 1
4 10
7 10
7 10
3 10
7 10
4 4
1 1
4 4
9 10
3 3
6 1
1 1
10 10
4 4
9 10
10 9
10 3
7 10
9 10
1 1
2 1
3 3
6 1
10 10
6 6
9 9
36 / 62 = 58.064516129
Loss = 90.0


(36, 58.064516129032263, '90.0')

In [72]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras.utils import np_utils

from matplotlib import pyplot as plt

def cnn_train():
    
    temp = normalized_X.reshape(normalized_X.shape[0], no_of_columns, 1)
    
    model = Sequential()
    # 13 7 1 1 0.25 60 0.25 10 - 70%
    model.add(Convolution1D(52, 13, activation='tanh', input_shape=(no_of_columns,1)))
    model.add(Convolution1D(52, 7, activation='tanh', use_bias=True))
    model.add(Convolution1D(13, 3, activation='tanh', use_bias=True))
    # stride = 2 - 70
    # 20, 10, 17 op - 64
    
    model.add(MaxPooling1D(pool_size=(1)))
#     model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='tanh'))
    model.add(Dropout(0.25))
    
    # 0.4 70
    model.add(Dense(classes+1, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(temp, one_hot_labels, epochs=classes if classes > 10 else 10, batch_size=32)
    return model

In [73]:
def test_cnn(model):
    
    test_model = get_feature_vectors("test")
#     print(test_model.shape)

    test_X = numpy.copy(test_model[:, :no_of_columns])
#     print(test_X.shape)

    normalized_test_X = (test_X - mean) / std_deviation
#     print(normalized_test_X.shape)

    test_Y = numpy.copy(test_model[:, no_of_columns:])
#     print(test_Y.shape)
    test_labels = np_utils.to_categorical(test_Y, num_classes=classes+1)
    
    test_X = test_X.reshape(test_X.shape[0], no_of_columns, 1)
    normalized_test_X = normalized_test_X.reshape(normalized_test_X.shape[0], no_of_columns, 1)
    
    print(model.test_on_batch(test_X, test_labels, sample_weight=None))
    print(model.metrics_names)
    predictions = model.predict(test_X)

    b = [sum(predictions[current: current+test_frames]) for current in range(0, len(predictions), test_frames)]
    predicted_Y = []
    for row in b:
        predicted_Y.append(row.argmax(axis=0))

    # print(predicted_Y)
    # print(test_Y[::40].T)
    
    indices = numpy.argmax(predictions, axis=1)
    majority = []
    
    for i in range(0, len(indices), test_frames):
        majority.append(find_majority(indices[i:i + test_frames]))
        
#     majority = 
    for t, p, m in zip(test_Y[::test_frames].T[0], predicted_Y, majority):
        print(int(t), p, m[0])
    
#     for t, p in zip(test_Y.T[0], indices):
#         print(int(t), p)
    
    
    diff = predicted_Y - test_Y[::test_frames].T[0]
    maj_diff = numpy.array(majority)[:, 0] - test_Y[::test_frames].T[0]

    numerator = sum(x == 0 for x in diff)
    denominator = len(predicted_Y)
    
    numerator2 = sum(x == 0 for x in maj_diff)
    denominator2 = len(maj_diff)
    

    print("Accuracy prob_diff: {} of {} - {}".format(numerator, denominator, numerator/denominator))
    
    print("Accuracy maj_diff: {} of {} - {}".format(numerator2, denominator2, numerator2/denominator2))
    

In [74]:
cnn_model = cnn_train()
test_cnn(cnn_model)

Epoch 1/10
3200/3200 [==============================] - 6s 2ms/step - loss: 1.3397 - acc: 0.5181
Epoch 2/10
3200/3200 [==============================] - 5s 2ms/step - loss: 0.7730 - acc: 0.7031
Epoch 3/10
3200/3200 [==============================] - 5s 2ms/step - loss: 0.5806 - acc: 0.7844
Epoch 4/10
3200/3200 [==============================] - 5s 2ms/step - loss: 0.4489 - acc: 0.8322
Epoch 5/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.3290 - acc: 0.8891
Epoch 6/10
3200/3200 [==============================] - 6s 2ms/step - loss: 0.2351 - acc: 0.9262
Epoch 7/10
3200/3200 [==============================] - 4s 1ms/step - loss: 0.1646 - acc: 0.9594
Epoch 8/10
3200/3200 [==============================] - 4s 1ms/step - loss: 0.1277 - acc: 0.9663
Epoch 9/10
3200/3200 [==============================] - 4s 1ms/step - loss: 0.0922 - acc: 0.9788
Epoch 10/10
3200/3200 [==============================] - 4s 1ms/step - loss: 0.0745 - acc: 0.9838


[array(3.997187614440918, dtype=float32), array(0.3076612949371338, dtype=float32)]
['loss', 'acc']
6 6 6
2 7 7
4 10 10
9 7 7
2 1 1
3 7 7
7 7 7
3 7 7
1 1 1
1 1 1
7 7 7
10 7 7
6 6 6
10 7 7
9 7 7
1 1 1
1 7 7
3 7 7
4 10 10
1 1 1
9 7 7
4 10 10
4 10 7
2 7 7
6 1 1
2 1 1
9 7 7
10 7 7
7 7 7
3 10 7
7 7 7
2 2 2
10 7 7
6 7 7
3 7 7
6 1 1
4 7 7
7 7 7
7 7 7
3 9 7
7 7 7
4 10 10
1 1 1
4 7 7
9 7 7
3 7 7
6 1 1
1 1 1
10 7 7
4 10 10
9 7 7
10 7 7
10 7 7
7 7 7
9 7 7
1 1 1
2 1 1
3 10 10
6 1 1
10 7 7
6 6 6
9 9 9
Accuracy prob_diff: 20 of 62 - 0.3225806451612903
Accuracy maj_diff: 20 of 62 - 0.3225806451612903


In [ ]:
# test(model)
test_cnn(cnn_model)

In [12]:

import python_speech_features
python_speech_features.__file__

# print(predicted_Y)
# print(test_Y[::40].T)

# for t, p in zip(test_Y[::40].T[0], predicted_Y):
#     print (int(t), p)



'/home/rohan/anaconda3/lib/python3.6/site-packages/python_speech_features-0.6-py3.6.egg/python_speech_features/__init__.py'

In [13]:
# majority

# argmax_pred = numpy.argmax(predictions, axis=1)
# argmax_pred = argmax_pred.reshape((-1, 25))

# from scipy.stats import mode
# argmax_pred = mode(argmax_pred, axis=-1)[0]

# # print(argmax_pred)
# # print(test_Y)

# for t, p in zip(test_Y[::40].T[0], argmax_pred):
#     print (int(t), p)

In [24]:
def find_majority(k):
    myMap = {}
    maximum = ( '', 0 ) # (occurring element, occurrences)
    for n in k:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1

        # Keep track of maximum on the go
        if myMap[n] > maximum[1]: maximum = (n,myMap[n])

    return maximum